In [ ]:
import chess
import chess.pgn
import csv
import chess.engine
from stockfish import Stockfish

stockfish = chess.engine.SimpleEngine.popen_uci("D:/Downloads/stockfish_15.1_win_x64_avx2/stockfish_15.1_win_x64_avx2/stockfish-windows-2022-x86-64-avx2.exe")
import pydirectory
  
# creating a virtual chessboard
board = chess.Board()

In [ ]:
from io import StringIO

# opening the pgn file
#pgn = open("lichess_elite_2018-01.pgn") #6561 games. 2mins @ 0.01 depth                11.49mins @ 0.1
# pgn = open("lichess_elite_2022-01.pgn")
pgn = open("lichess_elite_2019-12.pgn") # 59979 games. 105mins @ 0.1
#pgn = open("1.pgn")


In [ ]:
def stockfish_evaluation(board, time_limit = 0.1):

    result = stockfish.analyse(board, chess.engine.Limit(time=time_limit))

    scoreStore = result['score']

    scoreStore = str(scoreStore)
    scoreStore = scoreStore[11:scoreStore.find(" ")]

    scoreStore = scoreStore.replace("(","")
    scoreStore = scoreStore.replace(")","")
    scoreStore = scoreStore.replace(",","")
    return scoreStore


In [ ]:
#Put everything in a foor loop to loop through all games
k = 0
while True:

    # Read in each PGN game from the file
    game = chess.pgn.read_game(pgn)
    
    # If there are no more games, break out of the loop
    if game is None:
        break

    board = chess.Board()
    # username of the player playing with white
    white_username = game.headers['White']
    
    # username of the player playing with black
    black_username = game.headers['Black']
    # If white or black username is empty or contains ? skip the game
    if "?" in white_username:
        continue
    if "?" in black_username:
        continue

    #If the elo difference is greater than 50, skip the game
    if abs(int(game.headers['WhiteElo']) - int(game.headers['BlackElo'])) > 50:
        continue

    time_control = game.headers['TimeControl'] 

    game_result = game.headers['Result']  

    if game_result == "1-0":
        result = 1
    elif game_result == "0-1":
        result = 0
    else:
        result = 0.5
    # The move number for which we want the FEN
    move_number = 29

    i = 0
    CastleRightWhite = 1
    CastleRightBlack = 1

    for number, move in enumerate(game.mainline_moves()):

        if move == chess.Move.from_uci("e1g1"):
            CastleRightWhite = 2
        if move == chess.Move.from_uci("e1c1"):
            CastleRightWhite = 2
        if move == chess.Move.from_uci("e8g8"):
            CastleRightBlack = 2
        if move == chess.Move.from_uci("e8c8"):
            CastleRightBlack = 2
        
        # It copies the move played by each 
        # player on the virtual board
        board.push(move)
        i = i + 1


        #print that it is attempting black engine
        if i == move_number + 1:
            engineEval = stockfish_evaluation(board)
            #print("Engine evaluation: ", engineEval)
            break
        
        # Remember that number starts from 0
        if number == move_number:  
            break

    if i != move_number + 1:
        k = k + 1
        continue

    fen = board.fen()
    if fen.find("KQ") == -1 and CastleRightWhite == 1:
        CastleRightWhite = 0

    if fen.find("kq") == -1 and CastleRightBlack == 1:
        CastleRightBlack = 0

    fen = fen[0:fen.find(" ")]

    King = 0
    Queen = 9
    Rook = 5
    Bishop = 3
    Knight = 3
    Pawn = 1

    WhiteKing = fen.count("K")
    WhiteQueen = fen.count("Q")
    WhiteRook = fen.count("R")
    WhiteBishop = fen.count("B")
    WhiteKnight = fen.count("N")
    WhitePawn = fen.count("P")
    BlackKing = fen.count("k")
    BlackQueen = fen.count("q")
    BlackRook = fen.count("r")
    BlackBishop = fen.count("b")
    BlackKnight = fen.count("n")
    BlackPawn = fen.count("p")

    WhiteTotal = WhiteKing * King + WhiteQueen * Queen + WhiteRook * Rook + WhiteBishop * Bishop + WhiteKnight * Knight + WhitePawn * Pawn
    BlackTotal = BlackKing * King + BlackQueen * Queen + BlackRook * Rook + BlackBishop * Bishop + BlackKnight * Knight + BlackPawn * Pawn

    PointValueDifference = WhiteTotal - BlackTotal

    rookPair = 0
    if WhiteRook == 0 and BlackRook == 0:
        rookPair = 5
    elif WhiteRook == 1 and BlackRook == 0:
        rookPair = 6
    elif WhiteRook == 0 and BlackRook == 1:
        rookPair = 2
    elif WhiteRook == 1 and BlackRook == 1:
        rookPair = 4
    elif WhiteRook == 2 and BlackRook == 1:
        rookPair = 7
    elif WhiteRook == 1 and BlackRook == 2:
        rookPair = 1
    elif WhiteRook == 2 and BlackRook == 2:
        rookPair = 3
    elif WhiteRook == 2 and BlackRook == 0:
        rookPair = 1
    elif WhiteRook == 0 and BlackRook == 2:
        rookPair = 0

    bishopPair = 0
    if WhiteBishop == 0 and BlackBishop == 0:
        bishopPair = 5
    elif WhiteBishop == 1 and BlackBishop == 0:
        bishopPair = 6
    elif WhiteBishop == 0 and BlackBishop == 1:
        bishopPair = 2
    elif WhiteBishop == 1 and BlackBishop == 1:
        bishopPair = 4
    elif WhiteBishop == 2 and BlackBishop == 1:
        bishopPair = 7
    elif WhiteBishop == 1 and BlackBishop == 2:
        bishopPair = 1
    elif WhiteBishop == 2 and BlackBishop == 2:
        bishopPair = 3
    elif WhiteBishop == 2 and BlackBishop == 0:
        bishopPair = 1
    elif WhiteBishop == 0 and BlackBishop == 2:
        bishopPair = 0

    knightPair = 0
    if WhiteKnight == 0 and BlackKnight == 0:
        knightPair = 5
    elif WhiteKnight == 1 and BlackKnight == 0:
        knightPair = 6
    elif WhiteKnight == 0 and BlackKnight == 1:
        knightPair = 2
    elif WhiteKnight == 1 and BlackKnight == 1:
        knightPair = 4
    elif WhiteKnight == 2 and BlackKnight == 1:
        knightPair = 7
    elif WhiteKnight == 1 and BlackKnight == 2:
        knightPair = 1
    elif WhiteKnight == 2 and BlackKnight == 2:
        knightPair = 3
    elif WhiteKnight == 2 and BlackKnight == 0:
        knightPair = 1
    elif WhiteKnight == 0 and BlackKnight == 2:
        knightPair = 0

    #If white elo is between 0 and 1000, then print into a .csv file
    if int(game.headers['WhiteElo']) >= 0 and int(game.headers['WhiteElo']) <= 1000:
        # Open the .csv file in append mode
        with open('chess1.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([game.headers['WhiteElo'], game.headers['BlackElo'], len(list(board.legal_moves)), WhiteQueen + BlackQueen, WhitePawn + BlackPawn, bishopPair, knightPair, rookPair, WhiteTotal, BlackTotal, PointValueDifference, CastleRightWhite, CastleRightBlack, result, engineEval])


    #If white elo is between 1000 and 1500, then print into a .csv file
    if int(game.headers['WhiteElo']) > 1000 and int(game.headers['WhiteElo']) <= 1500:
        # Open the .csv file in append mode
        with open('chess2.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([game.headers['WhiteElo'], game.headers['BlackElo'], len(list(board.legal_moves)), WhiteQueen + BlackQueen, WhitePawn + BlackPawn, bishopPair, knightPair, rookPair, WhiteTotal, BlackTotal, PointValueDifference, CastleRightWhite, CastleRightBlack, result, engineEval])


    #If white elo is between 1500 and 2000, then print into a .csv file
    if int(game.headers['WhiteElo']) > 1500 and int(game.headers['WhiteElo']) <= 2000:
        # Open the .csv file in append mode
        with open('chess3.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([game.headers['WhiteElo'], game.headers['BlackElo'], len(list(board.legal_moves)), WhiteQueen + BlackQueen, WhitePawn + BlackPawn, bishopPair, knightPair, rookPair, WhiteTotal, BlackTotal, PointValueDifference, CastleRightWhite, CastleRightBlack, result, engineEval])

    #If white elo is between 2000 and 2250, then print into a .csv file
    if int(game.headers['WhiteElo']) > 2000 and int(game.headers['WhiteElo']) <= 2250:
        # Open the .csv file in append mode
        with open('chess4.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([game.headers['WhiteElo'], game.headers['BlackElo'], len(list(board.legal_moves)), WhiteQueen + BlackQueen, WhitePawn + BlackPawn, bishopPair, knightPair, rookPair, WhiteTotal, BlackTotal, PointValueDifference, CastleRightWhite, CastleRightBlack, result, engineEval])

    #If white elo is between 2250 and 2500, then print into a .csv file
    if int(game.headers['WhiteElo']) > 2250 and int(game.headers['WhiteElo']) <= 2500:
        # Open the .csv file in append mode
        with open('chess5.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([game.headers['WhiteElo'], game.headers['BlackElo'], len(list(board.legal_moves)), WhiteQueen + BlackQueen, WhitePawn + BlackPawn, bishopPair, knightPair, rookPair, WhiteTotal, BlackTotal, PointValueDifference, CastleRightWhite, CastleRightBlack, result, engineEval])

    #If white elo is above 2500, then print into a .csv file
    if int(game.headers['WhiteElo']) > 2500:
        # Open the .csv file in append mode
        with open('chess6.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([game.headers['WhiteElo'], game.headers['BlackElo'], len(list(board.legal_moves)), WhiteQueen + BlackQueen, WhitePawn + BlackPawn, bishopPair, knightPair, rookPair, WhiteTotal, BlackTotal, PointValueDifference, CastleRightWhite, CastleRightBlack, result, engineEval])

    k = k + 1
    #Print the game number
print("Game number: ", k)


Game number:  59979
